## Exploratory Data Analysis on COVID-19 cases in INDIA 

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# for gathering latitude and longitude
from geopy.geocoders import Nominatim

In [5]:
import folium

In [6]:
url = 'https://docs.google.com/spreadsheets/u/0/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
dfs = pd.read_html(url, encoding='utf8')

### State wise data

In [8]:
statewise_df = dfs[2].copy()
statewise_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1.0,State,NaN,Confirmed,Recovered,Deaths,Active,Last_Updated_Time
1,2.0,Total,NaN,1362,138,46,1178,31/03/2020 10:47:27
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,Maharashtra,NaN,243,39,10,194,31/03/2020 09:52:24
4,4.0,Kerala,NaN,234,20,2,212,31/03/2020 09:07:27


In [9]:
statewise_df.drop(['Unnamed: 0','Unnamed: 2','Unnamed: 7'],axis=1,inplace=True)
statewise_df.head()

,Unnamed: 1,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,State,Confirmed,Recovered,Deaths,Active
1,Total,1362,138,46,1178
2,NaN,NaN,NaN,NaN,NaN
3,Maharashtra,243,39,10,194
4,Kerala,234,20,2,212


In [10]:
#make the first row as column headers 
def col_headers(df):
    df.columns = df.iloc[0]
    df.drop(df.index[0],inplace=True)
    #statewise_df.head()
    return df

In [11]:
statewise_df = col_headers(statewise_df)

In [12]:
statewise_df.isna().sum()

0
State        1
Confirmed    1
Recovered    1
Deaths       1
Active       1
dtype: int64

In [13]:
statewise_df.dropna(axis=0,inplace=True)
statewise_df.reset_index(drop=True, inplace=True)
statewise_df.head()

,State,Confirmed,Recovered,Deaths,Active
0,Total,1362,138,46,1178
1,Maharashtra,243,39,10,194
2,Kerala,234,20,2,212
3,Delhi,97,6,2,89
4,Uttar Pradesh,101,17,0,84


In [14]:
statewise_df.dtypes

0
State        object
Confirmed    object
Recovered    object
Deaths       object
Active       object
dtype: object

In [15]:
statewise_df = statewise_df.astype({"Confirmed": 'int', "Recovered": 'int', "Deaths": 'int', "Active": 'int'})
statewise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
State        38 non-null object
Confirmed    38 non-null int32
Recovered    38 non-null int32
Deaths       38 non-null int32
Active       38 non-null int32
dtypes: int32(4), object(1)
memory usage: 1.0+ KB


In [17]:
toplot = statewise_df[statewise_df['Confirmed']>5]

In [119]:
fig = go.Figure(go.Bar(x=toplot.loc[1:,'State'], y=toplot.loc[1:,'Active'], name='Active'))
fig.add_trace(go.Bar(x=toplot.loc[1:,'State'], y=toplot.loc[1:,'Deaths'], name='Deaths'))
fig.add_trace(go.Bar(x=toplot.loc[1:,'State'], y=toplot.loc[1:,'Recovered'], name='Recovered'))

fig.update_layout(title='Covid-19 Case Statuses by State',barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':toplot.loc[1:,'State']}, xaxis_title="States",
    yaxis_title="Case Count")

fig.show()

In [120]:
labels = statewise_df.loc[1:,'State']
values =  statewise_df.loc[1:,'Confirmed']
fig = go.Figure(data=[go.Pie(labels=labels, values=values,hole=.3)])
fig.update_traces(textposition='inside')
fig.update_layout(title='Covid-19 Confirmed Cases percent by State',uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

### RAW PATIENT DATA

In [20]:
raw_data_df = dfs[0].copy()
raw_data_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1.0,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,Current Status,...,Source_2,Source_3,Backup Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2,KL-AL-P1,02/02/2020,NaN,NaN,Alappuzha,Alappuzha,Kerala,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,3,KL-KS-P1,03/02/2020,NaN,NaN,Kasaragod,Kasaragod,Kerala,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
raw_data_df = col_headers(raw_data_df)
raw_data_df.head()

,1.0,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,Current Status,...,Source_2,Source_3,Backup Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2,KL-AL-P1,02/02/2020,NaN,NaN,Alappuzha,Alappuzha,Kerala,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,3,KL-KS-P1,03/02/2020,NaN,NaN,Kasaragod,Kasaragod,Kerala,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,Recovered,...,https://economictimes.indiatimes.com/news/poli...,NaN,Travel history to Italy and Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
raw_data_df.shape

(2072, 26)

In [23]:
raw_data_df =  raw_data_df[['Patient Number','State Patient Number','Date Announced','Age Bracket','Gender','Detected District','Detected State','Current Status','Backup Notes']]
raw_data_df.isna().sum()

0
Patient Number           473
State Patient Number    1775
Date Announced           710
Age Bracket             1522
Gender                  1475
Detected District        952
Detected State           710
Current Status           710
Backup Notes            1711
dtype: int64

In [24]:
raw_data_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected District,Detected State,Current Status,Backup Notes
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Kerala,Recovered,Student from Wuhan
3,2,KL-AL-P1,02/02/2020,NaN,NaN,Alappuzha,Kerala,Recovered,Student from Wuhan
4,3,KL-KS-P1,03/02/2020,NaN,NaN,Kasaragod,Kerala,Recovered,Student from Wuhan
5,4,DL-P1,02/03/2020,45,M,East Delhi,Delhi,Recovered,Travel history to Italy and Austria


In [25]:
raw_data_df.dtypes

0
Patient Number          object
State Patient Number    object
Date Announced          object
Age Bracket             object
Gender                  object
Detected District       object
Detected State          object
Current Status          object
Backup Notes            object
dtype: object

In [26]:
print('Earlier values:',raw_data_df['Gender'].unique())
raw_data_df['Gender'].fillna('Data not available',inplace=True)
print('After filling NA values: ',raw_data_df['Gender'].unique())

Earlier values: [nan 'F' 'M']
After filling NA values:  ['Data not available' 'F' 'M']


In [27]:
new_df = raw_data_df.groupby(['Current Status','Gender'])['Patient Number'].count()
new_df = new_df.to_frame()
new_df.reset_index(inplace=True)
new_df.head()

,Current Status,Gender,Patient Number
0,Deceased,F,6
1,Deceased,M,14
2,Hospitalized,Data not available,744
3,Hospitalized,F,177
4,Hospitalized,M,366


In [123]:
#sns.countplot(raw_data_df['Current Status'], hue=raw_data_df['Gender'])

fig = px.bar(new_df, x="Current Status", y="Patient Number", color='Gender', barmode='group',height=400)
fig.update_layout(title='Patient Status by Gender',yaxis_title="Case Count")
fig.show()

In [29]:
raw_data_df.loc[926:929,'Age Bracket'] = -1

In [30]:
raw_data_df["Date Announced"] = raw_data_df["Date Announced"].astype('datetime64[ns]')
raw_data_df["Age Bracket"] = pd.to_numeric(raw_data_df["Age Bracket"])
raw_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2072 entries, 1 to 2072
Data columns (total 9 columns):
Patient Number          1599 non-null object
State Patient Number    297 non-null object
Date Announced          1362 non-null datetime64[ns]
Age Bracket             550 non-null float64
Gender                  2072 non-null object
Detected District       1120 non-null object
Detected State          1362 non-null object
Current Status          1362 non-null object
Backup Notes            361 non-null object
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 241.9+ KB


In [31]:
raw_data_df['Age Bracket'].fillna(-1,inplace=True)

In [32]:
raw_data_df['Age Groups'] = pd.cut(x=raw_data_df['Age Bracket'], bins=[-2,0, 19, 29, 39, 49, 59, 69, 79, 89, 99]).astype(str)
raw_data_df['Age Groups'].head()

1     (-2, 0]
2    (19, 29]
3     (-2, 0]
4     (-2, 0]
5    (39, 49]
Name: Age Groups, dtype: object

In [33]:
raw_data_df['Age Groups'].replace('(-2, 0]','Data not available',inplace=True)

In [34]:
age_df = raw_data_df.groupby(['Age Groups','Gender'])['Patient Number'].count()
age_df = age_df.to_frame()
age_df.reset_index(inplace=True)
age_df.head()

,Age Groups,Gender,Patient Number
0,"(0, 19]",Data not available,1
1,"(0, 19]",F,15
2,"(0, 19]",M,16
3,"(19, 29]",Data not available,7
4,"(19, 29]",F,40


In [35]:
age_df.dtypes

Age Groups        object
Gender            object
Patient Number     int64
dtype: object

In [36]:
age_df.loc[age_df['Age Groups']=='Data not available','Patient Number'][23]

948

In [124]:
#plt.figure(figsize=(20,7))
#sns.countplot(raw_data_df.sort_values('age_bins').loc[1:,'age_bins'])
#plt.title('Age Groups affected')
#plt.xlabel('Age Groups')
print("There are {} patients whose Age data is not Available.\nThe Graph has been prepared for the patients whose Age data is available.".format(age_df.loc[age_df['Age Groups']=='Data not available','Patient Number'][23]+age_df.loc[age_df['Age Groups']=='Data not available','Patient Number'][24]+age_df.loc[age_df['Age Groups']=='Data not available','Patient Number'][25]))
fig = px.bar(age_df[age_df['Age Groups'] !='Data not available'], x="Age Groups", y="Patient Number", color='Gender', barmode='group',
             height=400)
fig.update_layout(title='COVID-19 affected Age Groups by Gender',yaxis_title="Case Count")
fig.show()

There are 1053 patients whose Age data is not Available.
The Graph has been prepared for the patients whose Age data is available.


In [78]:
len(raw_data_df['Detected District'].unique())

166

# Bubble Map 

In [38]:
demo = raw_data_df[['Patient Number','Detected District','Detected State']].copy()
demo.head()

,Patient Number,Detected District,Detected State
1,NaN,NaN,NaN
2,1,Thrissur,Kerala
3,2,Alappuzha,Kerala
4,3,Kasaragod,Kerala
5,4,East Delhi,Delhi


In [39]:
demo.dropna(axis=0,inplace=True)

In [40]:
demo.dtypes

0
Patient Number       object
Detected District    object
Detected State       object
dtype: object

In [111]:
lat = []
lon = []
geolocator = Nominatim(user_agent="GeocodeEarth")
for index,row in demo.iterrows():
    if row['Detected District'] != np.NaN:
        location = geolocator.geocode(row['Detected District']+', India',timeout=10)
        if location == None:
            if row['Detected State'] != np.NaN:
                location = geolocator.geocode(row['Detected State'],timeout=10)
    if location != None:
        #print(index, location.address)
        lat.append(location.latitude)
        lon.append(location.longitude)
    else:
        lat.append(np.NaN)
        lon.append(np.NaN)

In [112]:
demo.loc["Latitude"] = lat
demo.loc["Longitude"] = lon
demo.loc[].tail()

,Patient Number,Detected District,Detected State,Latitude,Longitude
1359,1358,Bareilly,Uttar Pradesh,28.457876,79.405571
1360,1359,Bareilly,Uttar Pradesh,28.457876,79.405571
1361,1360,Bareilly,Uttar Pradesh,28.457876,79.405571
1362,1361,Bareilly,Uttar Pradesh,28.457876,79.405571
1363,1362,Gopalganj,Bihar,26.420727,84.374064


In [115]:
forbubble = demo.groupby(['Detected State','Detected District','Latitude','Longitude'])['Patient Number'].count()
forbubble = forbubble.to_frame()
forbubble.reset_index(inplace=True)

In [116]:
forbubble = forbubble.rename(columns = {"Patient Number": "Case Count"})

In [118]:
forbubble

,Detected State,Detected District,Latitude,Longitude,Case Count
0,Andaman and Nicobar Islands,North and Middle Andaman,12.611239,92.831654,1
1,Andaman and Nicobar Islands,South Andaman,10.705690,92.487468,5
2,Andhra Pradesh,Chittoor,13.160105,79.155551,1
3,Andhra Pradesh,East Godavari,17.233496,81.722599,3
4,Andhra Pradesh,Guntur,16.291519,80.454159,4
...,...,...,...,...,...
160,West Bengal,Kalimpong,27.071690,88.472900,1
161,West Bengal,Kolkata,22.545412,88.356775,11
162,West Bengal,Nadia,23.484541,88.556763,5
163,West Bengal,North 24 Parganas,22.718280,88.642481,2


In [117]:
india_map = folium.Map(location=[20.5937,78.9629], zoom_start=5)
 
# I can add marker one by one on the map
#for i in range(0,len(data)):
#   folium.Circle(
#      location=[data.iloc[i]['lon'], data.iloc[i]['lat']],
#      popup=data.iloc[i]['name'],
#      radius=data.iloc[i]['value']*10000,
#      color='crimson',
#      fill=True,
#      fill_color='crimson'
#   ).add_to(m)
for lat, lon, name, count in zip(forbubble['Latitude'], forbubble['Longitude'], forbubble['Detected District'],forbubble['Case Count']):
    # only use the first neighborhood to label on the map
    label = '{} {} cases'.format(name, count)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=count/2,
        popup=label,
        color='crimson',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(india_map)
    
india_map

###  Cases reported

In [43]:
cases_df = dfs[3].copy()
cases_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1.0,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
1,2.0,30 January,1,1,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,31 January,0,1,0,0,0,0
4,4.0,01 February,0,1,0,0,0,0


In [44]:
cases_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [45]:
cases_df = col_headers(cases_df)
cases_df.head()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
1,30 January,1,1,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31 January,0,1,0,0,0,0
4,01 February,0,1,0,0,0,0
5,02 February,1,2,0,0,0,0


In [46]:
cases_df.isna().sum()

0
Date               12
Daily Confirmed    12
Total Confirmed    12
Daily Recovered    12
Total Recovered    12
Daily Deceased     12
Total Deceased     12
dtype: int64

In [47]:
cases_df.dropna(axis=0,inplace=True)
cases_df.reset_index(drop=True, inplace=True)
cases_df.head()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,30 January,1,1,0,0,0,0
1,31 January,0,1,0,0,0,0
2,01 February,0,1,0,0,0,0
3,02 February,1,2,0,0,0,0
4,03 February,1,3,0,0,0,0


In [48]:
cases_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
Date               61 non-null object
Daily Confirmed    61 non-null object
Total Confirmed    61 non-null object
Daily Recovered    61 non-null object
Total Recovered    61 non-null object
Daily Deceased     61 non-null object
Total Deceased     61 non-null object
dtypes: object(7)
memory usage: 3.5+ KB


In [49]:
cases_df = cases_df.astype({"Daily Confirmed": 'int', "Total Confirmed": 'int', "Daily Recovered": 'int',"Total Recovered":'int', "Daily Deceased": 'int',"Total Deceased":'int'})
cases_df.dtypes

0
Date               object
Daily Confirmed     int32
Total Confirmed     int32
Daily Recovered     int32
Total Recovered     int32
Daily Deceased      int32
Total Deceased      int32
dtype: object

In [50]:
cases_df['Date'] =  cases_df['Date'] + " 2020"

In [51]:
cases_df['Date'] = pd.to_datetime(cases_df['Date'], format='%d %B %Y')

In [52]:
cases_df.set_index('Date', inplace=True)

In [128]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Total Confirmed'],
                name="Total Confirmed",
                line_color='blue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Daily Confirmed'],
                name="Daily Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Total Recovered'],
                name="Total Recovered",
                line_color='lightgreen',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Daily Recovered'],
                name="Daily Recovered",
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Total Deceased'],
                name="Total Deceased",
                line_color='red',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Daily Deceased'],
                name="Daily Deceased",
                line_color='orange',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2020-01-25','2020-04-01'],
                  title_text="Number of cases- Confirmed, Recovered,Deceased",
                  xaxis_title="Date",
                  yaxis_title="Case Count")
fig.show()

In [126]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Daily Confirmed'],
                name="Daily Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Daily Recovered'],
                name="Daily Recovered",
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Daily Deceased'],
                name="Daily Deceased",
                line_color='red',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2020-01-25','2020-04-01'],
                  title_text="Daily number of cases- Confirmed, Recovered,Deceased",
                  xaxis_title="Date",
                  yaxis_title="Case Count")
fig.show()

In [127]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Total Confirmed'],
                name="Total Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Total Recovered'],
                name="Total Recovered",
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=cases_df.index,
                y=cases_df['Total Deceased'],
                name="Total Deceased",
                line_color='red',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2020-01-25','2020-04-01'],
                  title_text="Total number of cases- Confirmed, Recovered,Deceased",
                  xaxis_title="Date",
                  yaxis_title="Case Count")
fig.show()